# Multi-Tenor CDX Tranche Pricing & Correlation Analysis

This notebook implements pricing of CDX.NA.IG.45 tranches across multiple maturities (1Y, 2Y, 3Y, 5Y, 7Y, 10Y) using:
1. **Gaussian Copula Model**: One-factor Li (2000) model
2. **G-VG Copula Model**: Mixed Gaussian-Variance Gamma copula

We analyze:
- Correlation term structure
- Base correlation surface
- Model performance across different tenors

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.optimize import minimize_scalar, minimize
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Load Multi-Tenor Data

In [ ]:
# Load constituent CDS spreads
constituents = pd.read_csv('../data/cdx_constituents_multi_tenor.csv')

# Load multi-tenor tranche market data
with open('../data/cdx_market_data_multi_tenor.json', 'r') as f:
    market_data_multi = json.load(f)

# Load OIS curve
ois_curve = pd.read_csv('../data/ois_curve.csv')

print("="*60)
print("DATA LOADED")
print("="*60)
print(f"\nConstituents: {len(constituents)} companies")
print(f"Tenors available: {list(market_data_multi.keys())}")
print(f"\nTenor spreads available for each company:")
print([col for col in constituents.columns if 'Yr' in str(col) or 'Mo' in str(col)])
print(f"\nOIS Curve points: {len(ois_curve)}")
print("\nFirst 3 constituents:")
print(constituents[['Company', 'Recovery rate', 'Running Spread', '1 Yr', '2 Yr', '3 Yr', '5 Yr', '7 Yr', '10 Yr']].head(3))

## 2. Core Pricing Functions

### 2.1 Survival Probability Bootstrap

In [ ]:
def get_discount_factor(t, ois_curve_df):
    """Get discount factor at time t using OIS curve"""
    # Parse tenor strings to years
    def tenor_to_years(tenor):
        if 'W' in tenor:
            return int(tenor.replace('W', '')) / 52
        elif 'M' in tenor and 'Y' not in tenor:
            return int(tenor.replace('M', '')) / 12
        elif 'Y' in tenor:
            return int(tenor.replace('Y', ''))
        return 0
    
    ois_curve_df['Years'] = ois_curve_df['Tenor'].apply(tenor_to_years)
    ois_curve_df = ois_curve_df.sort_values('Years')
    
    # Linear interpolation of rates
    interp_func = interp1d(ois_curve_df['Years'], ois_curve_df['Mid_Yield']/100, 
                           kind='linear', fill_value='extrapolate')
    rate = float(interp_func(t))
    
    return np.exp(-rate * t)

def bootstrap_survival_probability(spread, running_spread, recovery, maturity, ois_curve_df, dt=0.25):
    """Bootstrap survival probability from CDS spread"""
    times = np.arange(dt, maturity + dt, dt)
    Q = np.ones(len(times))  # Survival probabilities
    
    # Convert spread to decimal
    spread_dec = spread / 10000
    running_dec = running_spread / 10000
    
    for i, t in enumerate(times):
        # Discount factor
        df = get_discount_factor(t, ois_curve_df)
        
        # RPV01 (premium leg)
        rpv01 = sum([get_discount_factor(times[j], ois_curve_df) * dt * Q[j] 
                     for j in range(i)])
        
        # Default leg (loss given default)
        lgd = 1 - recovery
        default_leg = sum([get_discount_factor(times[j], ois_curve_df) * dt * lgd * 
                          (Q[j-1] - Q[j] if j > 0 else 1 - Q[j])
                          for j in range(i)])
        
        # Solve for Q[i] using CDS pricing equation
        # spread * RPV01 = LGD * sum(DF * dQ)
        if i == 0:
            Q[i] = max(0.01, 1 - (spread_dec * dt * df) / (lgd * df * dt))
        else:
            prev_pv = spread_dec * rpv01 - default_leg
            Q[i] = max(0.01, Q[i-1] - (spread_dec * df * dt - prev_pv) / (lgd * df * dt))
    
    # Create interpolation function
    return interp1d(times, Q, kind='linear', fill_value='extrapolate')

print("Bootstrap function defined successfully")

### 2.2 Gaussian Copula Loss Distribution

In [ ]:
def gaussian_copula_loss_distribution(survival_probs, recoveries, correlation, maturity, M=1000):
    """
    Calculate loss distribution using Gaussian copula (Large Homogeneous Portfolio approximation)
    
    Args:
        survival_probs: List of survival probability functions
        recoveries: List of recovery rates
        correlation: Asset correlation (rho)
        maturity: Time horizon
        M: Number of grid points for market factor
    
    Returns:
        loss_grid: Loss levels (as fraction of portfolio)
        loss_dist: Probability distribution of losses
    """
    N = len(survival_probs)
    lgd = np.array([1 - r for r in recoveries]) / N  # Loss given default per name
    
    # Market factor grid
    z_grid = np.linspace(-5, 5, M)
    dz = z_grid[1] - z_grid[0]
    
    # Calculate conditional default probabilities for each market factor realization
    loss_dist_list = []
    
    for z in z_grid:
        # Conditional default probability for each name given market factor Z
        cond_default_probs = []
        for i in range(N):
            Q_T = max(0.001, min(0.999, survival_probs[i](maturity)))
            threshold = norm.ppf(1 - Q_T)  # Default threshold
            
            # Conditional default probability: P(ε < threshold | Z=z)
            cond_prob = norm.cdf((threshold - np.sqrt(correlation) * z) / np.sqrt(1 - correlation))
            cond_default_probs.append(cond_prob)
        
        # Expected loss given market factor z (LHP approximation)
        expected_loss = sum([lgd[i] * cond_default_probs[i] for i in range(N)])
        
        # Probability of this market factor
        prob_z = norm.pdf(z) * dz
        
        loss_dist_list.append((expected_loss, prob_z))
    
    # Aggregate loss distribution
    loss_grid = np.array([l[0] for l in loss_dist_list])
    loss_prob = np.array([l[1] for l in loss_dist_list])
    
    return loss_grid, loss_prob

print("Gaussian copula loss distribution function defined")

### 2.3 Tranche Pricing

In [ ]:
def price_tranche(loss_grid, loss_prob, attachment, detachment, recovery=0.4, 
                   running_spread=500, ois_curve_df=None, maturity=5.0, dt=0.25):
    """
    Price a CDO tranche given loss distribution
    
    Returns:
        fair_spread: Fair running spread in bps
        upfront: Upfront payment as % of tranche notional
    """
    tranche_size = detachment - attachment
    times = np.arange(dt, maturity + dt, dt)
    
    # Calculate tranche loss for each scenario
    tranche_losses = np.maximum(0, np.minimum(loss_grid - attachment, tranche_size)) / tranche_size
    
    # Expected tranche loss (default leg)
    expected_loss = np.sum(tranche_losses * loss_prob)
    
    # Calculate RPV01 (premium leg)
    rpv01 = 0
    for t in times:
        df = get_discount_factor(t, ois_curve_df)
        # Expected tranche survival = 1 - expected loss up to time t
        expected_survival = 1 - expected_loss * (t / maturity)  # Simplified
        rpv01 += df * dt * expected_survival
    
    # Fair spread: Expected Loss / RPV01
    df_maturity = get_discount_factor(maturity, ois_curve_df)
    fair_spread = (expected_loss * df_maturity / rpv01) * 10000  # in bps
    
    # Upfront payment
    upfront_pct = ((fair_spread - running_spread) / 10000) * rpv01 * 100  # as percentage
    
    return fair_spread, upfront_pct, expected_loss

print("Tranche pricing function defined")

## 3. Gaussian Copula Model - Multi-Tenor Pricing

We will price all 6 tenors (1Y, 2Y, 3Y, 5Y, 7Y, 10Y) and calibrate correlation for each.

In [ ]:
def calibrate_gaussian_correlation_for_tenor(tenor, constituents_df, market_data, ois_curve_df):
    """Calibrate Gaussian copula correlation for a specific tenor"""
    print(f"\nCalibrating Gaussian Copula for {tenor}...")
    
    maturity = float(tenor.replace('Y', ''))
    spread_col = tenor.replace('Y', ' Yr')
    
    # Bootstrap survival probabilities
    survival_probs = []
    recoveries = []
    
    for idx, row in constituents_df.iterrows():
        recovery = row['Recovery rate']
        running_spread = row['Running Spread']
        cds_spread = row[spread_col]
        
        try:
            surv_prob = bootstrap_survival_probability(
                cds_spread, running_spread, recovery, maturity, ois_curve_df
            )
            survival_probs.append(surv_prob)
            recoveries.append(recovery)
        except:
            survival_probs.append(lambda t: np.exp(-cds_spread/10000 * t / (1-recovery)))
            recoveries.append(recovery)
    
    market_tranches = market_data[tenor]
    
    def objective(rho):
        rho = max(0.01, min(0.99, rho))
        loss_grid, loss_prob = gaussian_copula_loss_distribution(
            survival_probs, recoveries, rho, maturity
        )
        
        total_error = 0
        tranches = [
            (0.00, 0.03, market_tranches['equity_0_3_running']),
            (0.03, 0.07, market_tranches['mezz_3_7']),
            (0.07, 0.10, market_tranches['mezz_7_10']),
            (0.10, 0.15, market_tranches['senior_10_15']),
            (0.15, 1.00, market_tranches['senior_15_100'])
        ]
        
        for attach, detach, mkt_spread in tranches:
            try:
                model_spread, _, _ = price_tranche(
                    loss_grid, loss_prob, attach, detach, 
                    running_spread=500 if attach == 0 else 100,
                    ois_curve_df=ois_curve_df, maturity=maturity
                )
                total_error += (model_spread - mkt_spread) ** 2
            except:
                total_error += 1e6
        
        return total_error
    
    result = minimize_scalar(objective, bounds=(0.1, 0.8), method='bounded')
    optimal_rho = result.x
    
    print(f"  Optimal correlation: {optimal_rho:.4f} ({optimal_rho*100:.2f}%)")
    return optimal_rho, survival_probs, recoveries

print("Calibration function defined")

### 3.1 Calibrate for All Tenors

In [ ]:
tenors = ['1Y', '2Y', '3Y', '5Y', '7Y', '10Y']
gaussian_results = {}

for tenor in tenors:
    optimal_rho, surv_probs, recoveries = calibrate_gaussian_correlation_for_tenor(
        tenor, constituents, market_data_multi, ois_curve
    )
    gaussian_results[tenor] = {
        'correlation': optimal_rho,
        'survival_probs': surv_probs,
        'recoveries': recoveries
    }

print("\n" + "="*60)
print("GAUSSIAN COPULA - CORRELATION TERM STRUCTURE")
print("="*60)
for tenor in tenors:
    print(f"{tenor}: ρ = {gaussian_results[tenor]['correlation']:.4f}")